In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
import datetime
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
from Cleanpipeline import chennai_house_df
import sklearn

   PRT_ID        AREA  INT_SQFT  DATE_SALE  DIST_MAINROAD  N_BEDROOM  \
0  P03210  Karapakkam      1004 2011-05-04            131        1.0   
1  P09411  Anna Nagar      1986 2006-12-19             26        2.0   
2  P01812      Adayar       909 2012-02-04             70        1.0   
3  P05346   Velachery      1855 2010-03-13             14        3.0   
4  P06210  Karapakkam      1226 2009-10-05             84        1.0   

   N_BATHROOM  N_ROOM SALE_COND PARK_FACIL  ... SALE_DAY_NAME SALE_QUARTER  \
0         1.0       3  AbNormal        Yes  ...     Wednesday            2   
1         1.0       5  AbNormal         No  ...       Tuesday            4   
2         1.0       3  AbNormal        Yes  ...      Saturday            1   
3         2.0       5    Family         No  ...      Saturday            1   
4         1.0       3  AbNormal        Yes  ...        Monday            4   

  BUILD_YEAR BUILD_MONTH BUILD_MONTH_NAME  BUILD_DAY  BUILD_DAY_OF_WEEK  \
0       1967           

In [2]:
df = chennai_house_df.copy()

In [3]:
# Select required columns

df = df[['AREA','BUILDING_AGE', 'INT_SQFT', 'DATE_SALE', 'DIST_MAINROAD', 'N_BEDROOM',
       'N_BATHROOM', 'N_ROOM', 'SALE_COND', 'PARK_FACIL', 'DATE_BUILD',
       'BUILDTYPE', 'UTILITY_AVAIL', 'STREET', 'MZZONE', 'QS_ROOMS',
       'QS_BATHROOM', 'QS_BEDROOM', 'QS_OVERALL', 'SALES_PRICE']]

## Categorical column Target encoding

In [4]:
# Filter out categorical column
X_cat = df.select_dtypes('object')
X_cat

,AREA,SALE_COND,PARK_FACIL,BUILDTYPE,UTILITY_AVAIL,STREET,MZZONE
0,Karapakkam,AbNormal,Yes,Commercial,AllPub,Paved,A
1,Anna Nagar,AbNormal,No,Commercial,AllPub,Gravel,RH
2,Adayar,AbNormal,Yes,Commercial,ELO,Gravel,RL
3,Velachery,Family,No,Others,NoSewr,Paved,I
4,Karapakkam,AbNormal,Yes,Others,AllPub,Gravel,C
...,...,...,...,...,...,...,...
7098,Karapakkam,AdjLand,No,Others,ELO,No Access,RM
7099,Velachery,Family,Yes,Others,NoSeWa,No Access,RH
7100,Velachery,Normal Sale,No,House,NoSeWa,Gravel,I
7101,Karapakkam,Partial,Yes,Commercial,ELO,Paved,RL


In [5]:
order_col = [list(df.groupby(i)['SALES_PRICE'].mean().sort_values(ascending=False).index) for i in X_cat.columns ]

In [6]:
# encode the columns
from sklearn.preprocessing import OrdinalEncoder
order = OrdinalEncoder(categories=order_col,dtype='int64')
X_cat_En = pd.DataFrame(order.fit_transform(X_cat),columns=order.get_feature_names_out())
X_cat_En

,AREA,SALE_COND,PARK_FACIL,BUILDTYPE,UTILITY_AVAIL,STREET,MZZONE
0,6,2,0,0,0,1,5
1,1,2,1,0,0,0,2
2,5,2,0,0,3,0,1
3,3,3,1,1,1,1,3
4,6,2,0,1,0,0,4
...,...,...,...,...,...,...,...
7098,6,0,1,1,3,2,0
7099,3,3,0,1,2,2,2
7100,3,1,1,2,2,0,3
7101,6,4,0,0,3,1,1


In [7]:
# separate 
X_cat_En1 = df[['N_BEDROOM','N_BATHROOM','N_ROOM']]
X_cat_En1

,N_BEDROOM,N_BATHROOM,N_ROOM
0,1.0,1.0,3
1,2.0,1.0,5
2,1.0,1.0,3
3,3.0,2.0,5
4,1.0,1.0,3
...,...,...,...
7098,1.0,1.0,2
7099,3.0,2.0,5
7100,2.0,1.0,4
7101,1.0,1.0,2


In [8]:
y = df['SALES_PRICE']

Numerical columns encoding using standardization

In [9]:
X_num = df.select_dtypes(['int32','int64','float'])
X_num.drop(columns=['N_BEDROOM','N_BATHROOM','N_ROOM'],inplace=True)

In [10]:
X_num.corr()['SALES_PRICE'].sort_values(ascending=False)

SALES_PRICE      1.000000
INT_SQFT         0.612026
QS_ROOMS         0.021960
QS_OVERALL       0.019784
DIST_MAINROAD    0.018679
QS_BEDROOM       0.018430
QS_BATHROOM     -0.011361
BUILDING_AGE    -0.118644
Name: SALES_PRICE, dtype: float64

In [11]:
X_num = X_num[['INT_SQFT','BUILDING_AGE']]

In [12]:
from sklearn.preprocessing import StandardScaler
Scale = StandardScaler()
X_num_en = pd.DataFrame(Scale.fit_transform(X_num),columns=Scale.get_feature_names_out())
X_num_en

,INT_SQFT,BUILDING_AGE
0,-0.826573,1.599627
1,1.320099,-1.076033
2,-1.034245,-0.346308
3,1.033731,-0.184146
4,-0.341276,0.464498
...,...,...
7098,-1.714097,2.005030
7099,1.125543,-1.238194
7100,0.506900,0.302337
7101,-1.300939,0.626660


In [13]:
df

,AREA,BUILDING_AGE,INT_SQFT,DATE_SALE,DIST_MAINROAD,N_BEDROOM,N_BATHROOM,N_ROOM,SALE_COND,PARK_FACIL,DATE_BUILD,BUILDTYPE,UTILITY_AVAIL,STREET,MZZONE,QS_ROOMS,QS_BATHROOM,QS_BEDROOM,QS_OVERALL,SALES_PRICE
0,Karapakkam,44,1004,2011-05-04,131,1.0,1.0,3,AbNormal,Yes,1967-05-15,Commercial,AllPub,Paved,A,4.0,3.9,4.9,4.330,7600000
1,Anna Nagar,11,1986,2006-12-19,26,2.0,1.0,5,AbNormal,No,1995-12-22,Commercial,AllPub,Gravel,RH,4.9,4.2,2.5,3.765,21717770
2,Adayar,20,909,2012-02-04,70,1.0,1.0,3,AbNormal,Yes,1992-02-09,Commercial,ELO,Gravel,RL,4.1,3.8,2.2,3.090,13159200
3,Velachery,22,1855,2010-03-13,14,3.0,2.0,5,Family,No,1988-03-18,Others,NoSewr,Paved,I,4.7,3.9,3.6,4.010,9630290
4,Karapakkam,30,1226,2009-10-05,84,1.0,1.0,3,AbNormal,Yes,1979-10-13,Others,AllPub,Gravel,C,3.0,2.5,4.1,3.290,7406250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7098,Karapakkam,49,598,2011-01-03,51,1.0,1.0,2,AdjLand,No,1962-01-15,Others,ELO,No Access,RM,3.0,2.2,2.4,2.520,5353000
7099,Velachery,9,1897,2004-04-08,52,3.0,2.0,5,Family,Yes,1995-04-11,Others,NoSeWa,No Access,RH,3.6,4.5,3.3,3.920,10818480
7100,Velachery,28,1614,2006-08-25,152,2.0,1.0,4,Normal Sale,No,1978-09-01,House,NoSeWa,Gravel,I,4.3,4.2,2.9,3.840,8351410
7101,Karapakkam,32,787,2009-08-03,40,1.0,1.0,2,Partial,Yes,1977-08-11,Commercial,ELO,Paved,RL,4.6,3.8,4.1,4.160,8507000


In [14]:
pd.concat([X_cat_En,X_cat_En1,X_num_en],axis=1)

,AREA,SALE_COND,PARK_FACIL,BUILDTYPE,UTILITY_AVAIL,STREET,MZZONE,N_BEDROOM,N_BATHROOM,N_ROOM,INT_SQFT,BUILDING_AGE
0,6,2,0,0,0,1,5,1.0,1.0,3,-0.826573,1.599627
1,1,2,1,0,0,0,2,2.0,1.0,5,1.320099,-1.076033
2,5,2,0,0,3,0,1,1.0,1.0,3,-1.034245,-0.346308
3,3,3,1,1,1,1,3,3.0,2.0,5,1.033731,-0.184146
4,6,2,0,1,0,0,4,1.0,1.0,3,-0.341276,0.464498
...,...,...,...,...,...,...,...,...,...,...,...,...
7098,6,0,1,1,3,2,0,1.0,1.0,2,-1.714097,2.005030
7099,3,3,0,1,2,2,2,3.0,2.0,5,1.125543,-1.238194
7100,3,1,1,2,2,0,3,2.0,1.0,4,0.506900,0.302337
7101,6,4,0,0,3,1,1,1.0,1.0,2,-1.300939,0.626660


## Pipeline creation using Sklearn

In [15]:
X_cat

,AREA,SALE_COND,PARK_FACIL,BUILDTYPE,UTILITY_AVAIL,STREET,MZZONE
0,Karapakkam,AbNormal,Yes,Commercial,AllPub,Paved,A
1,Anna Nagar,AbNormal,No,Commercial,AllPub,Gravel,RH
2,Adayar,AbNormal,Yes,Commercial,ELO,Gravel,RL
3,Velachery,Family,No,Others,NoSewr,Paved,I
4,Karapakkam,AbNormal,Yes,Others,AllPub,Gravel,C
...,...,...,...,...,...,...,...
7098,Karapakkam,AdjLand,No,Others,ELO,No Access,RM
7099,Velachery,Family,Yes,Others,NoSeWa,No Access,RH
7100,Velachery,Normal Sale,No,House,NoSeWa,Gravel,I
7101,Karapakkam,Partial,Yes,Commercial,ELO,Paved,RL


In [16]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

In [18]:
cat_fea =X_cat.columns
num_fea =X_num.columns
num_fea_dis = ['N_BEDROOM','N_BATHROOM','N_ROOM']

In [19]:
num_transfrom = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
num_transfrom

,steps,"[('imputer', ...), ('scaler', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'median'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,copy,True


In [21]:
cat_transform = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(categories=order_col,dtype='int64'))
])
cat_transform

,steps,"[('imputer', ...), ('encoder', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'most_frequent'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,categories,"[['T Nagar', 'Anna Nagar', ...], ['AdjLand', 'Normal Sale', ...], ...]"


In [22]:
preprocess = ColumnTransformer(
    transformers=[
        ("num", num_transfrom,num_fea),
        ("cat", cat_transform,cat_fea)
    ]
)
preprocess

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [23]:
pline = Pipeline(steps=[
    ('preprocess',preprocess)
])
pline

,steps,"[('preprocess', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [28]:
num_fea_dis

['N_BEDROOM', 'N_BATHROOM', 'N_ROOM']

In [30]:
x = df[['AREA', 'SALE_COND', 'PARK_FACIL', 'BUILDTYPE', 'UTILITY_AVAIL',
       'STREET', 'MZZONE','INT_SQFT', 'BUILDING_AGE','N_BEDROOM', 'N_BATHROOM', 'N_ROOM']]

In [31]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(5327, 12)
(1776, 12)
(5327,)
(1776,)


In [33]:
pd.DataFrame(pline.fit_transform(x_train),columns=pline.get_feature_names_out()).head()

,num__INT_SQFT,num__BUILDING_AGE,cat__AREA,cat__SALE_COND,cat__PARK_FACIL,cat__BUILDTYPE,cat__UTILITY_AVAIL,cat__STREET,cat__MZZONE
0,-0.778397,0.627685,4.0,0.0,0.0,0.0,0.0,1.0,2.0
1,-0.752219,1.196836,4.0,0.0,1.0,2.0,0.0,0.0,0.0
2,0.604658,0.708993,1.0,3.0,0.0,2.0,2.0,2.0,1.0
3,0.641743,-1.567611,1.0,4.0,0.0,2.0,2.0,2.0,2.0
4,-0.464265,1.115529,4.0,2.0,1.0,0.0,0.0,1.0,2.0


In [34]:
pline.transform(x_test)

array([[ 0.48685822, -0.59192354,  6.        , ...,  3.        ,
         0.        ,  5.        ],
       [-0.52752734, -0.8358453 ,  4.        , ...,  3.        ,
         2.        ,  2.        ],
       [ 0.77917579, -1.32368884,  1.        , ...,  0.        ,
         2.        ,  2.        ],
       ...,
       [-1.58772387, -0.34800177,  5.        , ...,  3.        ,
         2.        ,  0.        ],
       [-1.48955753,  1.27814334,  4.        , ...,  3.        ,
         1.        ,  0.        ],
       [-1.15797343, -0.51061628,  5.        , ...,  3.        ,
         0.        ,  4.        ]], shape=(1776, 9))